# IA Aumentada: Entendendo e praticando otimização com OR-Tools

## Referências:
* [Google OR-Tools](https://developers.google.com/optimization/introduction)

## Instalar libs

In [1]:
! pip install ortools

## Importar bibliotecas

In [2]:
import pandas as pd
from ortools.sat.python import cp_model

## Aula 1

In [3]:
modelo = cp_model.CpModel()

In [4]:
gate_1 = modelo.NewIntVar(1, 5, 'gate_1')
gate_2= modelo.NewIntVar(1, 5, 'gate_2')

In [5]:
modelo.Add(gate_1 != 1)
modelo.Add(gate_1 > 4)

In [6]:
bonus = modelo.NewIntVar(0, 1, 'bonus')
modelo.Add(gate_2 > 10).OnlyEnforceIf(bonus)
modelo.Add(gate_2 <= 10).OnlyEnforceIf(bonus.Not())

In [7]:
modelo.Maximize(bonus)

In [8]:
solver = cp_model.CpSolver()
status = solver.Solve(modelo)

print('Status do solucionador: \t%s' % solver.StatusName(status))
print('Gate 1: \t\t%d' % solver.Value(gate_1))
print('Gate 2: \t\t%d' % solver.Value(gate_2))
print('Bonus: \t\t\t\t%d' % solver.Value(bonus))

Status do solucionador: 	OPTIMAL
Gate 1: 		5
Gate 2: 		1
Bonus: 				0


## Aula 2

* Problema: O mesmo gate com mais de uma aeronave

In [9]:
modelo = cp_model.CpModel()

gate_1 = modelo.NewIntVar(1, 5, 'gate_1')
gate_2 = modelo.NewIntVar(1, 5, 'gate_2')

solver = cp_model.CpSolver()
status = solver.Solve(modelo)

print('Status do solucionador: \t%s' % solver.StatusName(status))
print('Gate 1: \t\t\t%d' % solver.Value(gate_1))
print('Gate 2: \t\t\t%d' % solver.Value(gate_2))

Status do solucionador: 	OPTIMAL
Gate 1: 			1
Gate 2: 			1


* Solucao: Aplicar regra restritiva para diferenciar

In [10]:
modelo = cp_model.CpModel()

gate_1 = modelo.NewIntVar(1, 5, 'gate_1')
gate_2 = modelo.NewIntVar(1, 5, 'gate_2')

modelo.Add(gate_1 != gate_2)

solver = cp_model.CpSolver()
status = solver.Solve(modelo)

print('Status do solucionador: \t%s' % solver.StatusName(status))
print('Gate 1: \t\t\t%d' % solver.Value(gate_1))
print('Gate 2: \t\t\t%d' % solver.Value(gate_2))

Status do solucionador: 	OPTIMAL
Gate 1: 			2
Gate 2: 			1


* Criar functions

In [11]:
def create_gates(model, gates_total, capacity):
    return [model.NewIntVar(1, capacity, f'gate_{idx}') for idx in range(gates_total)]

In [12]:
def solver_model(model, solver, gates):
    status = solver.Solve(model)
    print('Status do solucionador: %s' % solver.StatusName(status))
    if status == cp_model.INFEASIBLE:
        print('Sem solucao')
        return
    for idx, gate in enumerate(gates):
        print('Gate %d: %d' % (idx, solver.Value(gate)))

In [13]:
def distinct_planes(model, gates):
    model.AddAllDifferent(gates)

In [14]:
scenarios = [[3, 3], # Deve haver solucao para 3 vagas e 3 aeronaves
             [2, 3], # Deve haver solucao para 3 vagas e 2 aeronaves
             [2, 2], # Deve haver solucao para 2 vagas e 2 aeronaves
             [2, 1]] # Nao deve haver solucao para 1 vaga e 2 aeronaves

for gates_total, capacity in scenarios:
    model = cp_model.CpModel()

    gates = create_gates(model, gates_total, capacity)

    distinct_planes(model, gates)

    solver_model(model, cp_model.CpSolver(), gates)

    print('\n')

Status do solucionador: OPTIMAL
Gate 0: 2
Gate 1: 3
Gate 2: 1


Status do solucionador: OPTIMAL
Gate 0: 1
Gate 1: 3


Status do solucionador: OPTIMAL
Gate 0: 1
Gate 1: 2


Status do solucionador: INFEASIBLE
Sem solucao




## Aula 3

* Problema: aeronaves nao estao pousando

In [15]:
def create_gates(model, gates_total, capacity):
    return [model.NewIntVar(0, capacity, f'gate_{idx}') for idx in range(gates_total)]

In [16]:
model = cp_model.CpModel()
gates = create_gates(model, 2, 2)

distinct_planes(model, gates)

solver_model(model, cp_model.CpSolver(), gates)

Status do solucionador: OPTIMAL
Gate 0: 0
Gate 1: 2


In [17]:
model = cp_model.CpModel()
gates = create_gates(model, 2, 1)

distinct_planes(model, gates)

solver_model(model, cp_model.CpSolver(), gates)

Status do solucionador: OPTIMAL
Gate 0: 0
Gate 1: 1


* Solucao: Aplicar regra que toda aeronave deve pousar

In [18]:
def all_planes_must_land(model, gates, capacity):
    variables = {}

    for i in range(1, capacity + 1):
        for j, gate in enumerate(gates):
            plane_i_j = model.NewBoolVar(f'plane_{i}_{j}')
            model.Add(gate == 1).OnlyEnforceIf(plane_i_j)
            model.Add(gate != 1).OnlyEnforceIf(plane_i_j.Not())
            variables[(i, j)] = plane_i_j
    
    for i in range(1, capacity + 1):
        model.AddExactlyOne([variables[i, j] for j in range(len(gates))])

In [19]:
model = cp_model.CpModel()
gates = create_gates(model, 2, 2)

distinct_planes(model, gates)
all_planes_must_land(model, gates, 2)

solver_model(model, cp_model.CpSolver(), gates)

Status do solucionador: OPTIMAL
Gate 0: 2
Gate 1: 1


In [20]:
model = cp_model.CpModel()
gates = create_gates(model, 3, 2)

distinct_planes(model, gates)
all_planes_must_land(model, gates, 2)

solver_model(model, cp_model.CpSolver(), gates)

Status do solucionador: OPTIMAL
Gate 0: 2
Gate 1: 0
Gate 2: 1


In [21]:
model = cp_model.CpModel()
gates = create_gates(model, 4, 2)

distinct_planes(model, gates)
all_planes_must_land(model, gates, 2)

solver_model(model, cp_model.CpSolver(), gates)

Status do solucionador: INFEASIBLE
Sem solucao


## Aula 4

In [22]:
class Gate:
    def __init__(self, large: bool):
        self.large = large

In [23]:
class Plane:
    def __init__(self, k: int, large: bool):
        self.k = k
        self.large = large

In [24]:
def remove_from_gate(model, gate, planes):
    for plane in planes:
        model.Add(gate != plane.k)

In [25]:
def large_plane_large_gate(model, gates, planes):
    large_planes = [plane for plane in planes if plane.large]

    for gate in gates:
        if not gate.large:
            remove_from_gate(model, gate.variable, large_planes)

In [26]:
def create_gates(model, gates, capacity):
    variables = []

    for i, gate in enumerate(gates):
        variable = model.NewIntVar(0, capacity, f'gate_{i}')
        gate.variable = variable
        variables.append(variable)
    
    return variables

In [27]:
def solver_model(model, solver, gates, planes):
    status = solver.Solve(model)
    print('Status do solucionador: %s' % solver.StatusName(status))
    if status == cp_model.INFEASIBLE:
        print('Sem solucao')
        return
    for gate in gates:
        value = solver.Value(gate)
        if value == 0:
            print(f'{gate} sem aeronave')
        else:
            plane = planes[value - 1]
            print(f'{gate} com aeronave {value} | Grande: {plane.large}')

In [28]:
planes = [Plane(1, True),
          Plane(2, False)]

gates = [Gate(False),
         Gate(False),
         Gate(True)]

model = cp_model.CpModel()

gates_vars = create_gates(model, gates, len(planes))

distinct_planes(model, gates_vars)
all_planes_must_land(model, gates_vars, len(planes))
large_plane_large_gate(model, gates, planes)

solver_model(model, cp_model.CpSolver(), gates_vars, planes)

Status do solucionador: OPTIMAL
gate_0 sem aeronave
gate_1 com aeronave 2 | Grande: False
gate_2 com aeronave 1 | Grande: True


In [29]:
planes = [Plane(1, True),
          Plane(2, True)]

gates = [Gate(True),
         Gate(True),
         Gate(False)]

model = cp_model.CpModel()

gates_vars = create_gates(model, gates, len(planes))

distinct_planes(model, gates_vars)
all_planes_must_land(model, gates_vars, len(planes))
large_plane_large_gate(model, gates, planes)

solver_model(model, cp_model.CpSolver(), gates_vars, planes)

Status do solucionador: OPTIMAL
gate_0 com aeronave 2 | Grande: True
gate_1 com aeronave 1 | Grande: True
gate_2 sem aeronave


In [30]:
planes = [Plane(1, True),
          Plane(2, True)]

gates = [Gate(True),
         Gate(False),
         Gate(False)]

model = cp_model.CpModel()

gates_vars = create_gates(model, gates, len(planes))

distinct_planes(model, gates_vars)
all_planes_must_land(model, gates_vars, len(planes))
large_plane_large_gate(model, gates, planes)

solver_model(model, cp_model.CpSolver(), gates_vars, planes)

Status do solucionador: INFEASIBLE
Sem solucao


## Aula 5

In [31]:
class Gate:
    def __init__(self, model, k, size, large):
        self.large = large
        self.variable = model.NewIntVar(0, size, f'gate_{k}')
        self.k = k
        self.neighbors = []
        self.receive_large_plane = model.NewBoolVar(f'gate_receive_large_plane_{k}')
        if not self.large:
            model.Add(self.receive_large_plane == 0)

In [32]:
def solver_model(model, solver, gates, planes):
    status = solver.Solve(model)
    print('Status do solucionador: %s' % solver.StatusName(status))
    if status == cp_model.INFEASIBLE:
        print('Sem solucao')
        return
    for gate in gates:
        value = solver.Value(gate.variable)
        if value == 0:
            print(f'{gate.variable} sem aeronave')
        else:
            plane = planes[value - 1]
            print(f'{gate.variable} com aeronave {value} | Grande: {plane.large}')

In [33]:
def large_plane_large_gate(model, gates, planes):
    large_planes = [plane for plane in planes if plane.large]
    for gate in gates:
        for plane in large_planes:
            model.Add(gate.variable != plane.k).OnlyEnforceIf(gate.receive_large_plane.Not())

In [34]:
def distinct_planes(model, gates):
    model.AddAllDifferent([gate.variable for gate in gates])

In [35]:
def all_planes_must_land(model, gates, capacity):
    variables = {}

    for i in range(1, capacity + 1):
        for j, gate in enumerate(gates):
            plane_i_j = model.NewBoolVar(f'plane_{i}_{j}')
            model.Add(gate.variable == 1).OnlyEnforceIf(plane_i_j)
            model.Add(gate.variable != 1).OnlyEnforceIf(plane_i_j.Not())
            variables[(i, j)] = plane_i_j
    
    for i in range(1, capacity + 1):
        model.AddExactlyOne([variables[i, j] for j in range(len(gates))])

In [36]:
def restrict_neighbors(model, gates, planes):
    for gate in gates:
        if not gate.large:
            continue
        for neighbor in gate.neighbors:
            if neighbor.large:
                model.Add(gate.receive_large_plane == 0).OnlyEnforceIf(neighbor.receive_large_plane)

In [37]:
planes = [Plane(1, True),
          Plane(2, False),
          Plane(3, True)]

planes_total = len(planes)

model = cp_model.CpModel()

gates = [Gate(model, 1, planes_total, False),
         Gate(model, 2, planes_total, False),
         Gate(model, 3, planes_total, True),
         Gate(model, 4, planes_total, True)]

gates[2].neighbors = [gates[3]]

distinct_planes(model, gates)
all_planes_must_land(model, gates, planes_total)
restrict_neighbors(model, gates, planes)
large_plane_large_gate(model, gates, planes)

solver_model(model, cp_model.CpSolver(), gates, planes)

Status do solucionador: INFEASIBLE
Sem solucao


In [38]:
planes = [Plane(1, True),
          Plane(2, False),
          Plane(3, True)]

planes_total = len(planes)

model = cp_model.CpModel()

gates = [Gate(model, 1, planes_total, False),
         Gate(model, 2, planes_total, False),
         Gate(model, 3, planes_total, True),
         Gate(model, 4, planes_total, True)]

gates[2].neighbors = [gates[0]]

distinct_planes(model, gates)
all_planes_must_land(model, gates, planes_total)
restrict_neighbors(model, gates, planes)
large_plane_large_gate(model, gates, planes)

solver_model(model, cp_model.CpSolver(), gates, planes)

Status do solucionador: OPTIMAL
gate_1 com aeronave 2 | Grande: False
gate_2 sem aeronave
gate_3 com aeronave 3 | Grande: True
gate_4 com aeronave 1 | Grande: True


In [39]:
planes = [Plane(1, True),
          Plane(2, True),
          Plane(3, True)]

planes_total = len(planes)

model = cp_model.CpModel()

gates = [Gate(model, 1, planes_total, False),
         Gate(model, 2, planes_total, False),
         Gate(model, 3, planes_total, True),
         Gate(model, 4, planes_total, True)]

gates[2].neighbors = [gates[0]]

distinct_planes(model, gates)
all_planes_must_land(model, gates, planes_total)
restrict_neighbors(model, gates, planes)
large_plane_large_gate(model, gates, planes)

solver_model(model, cp_model.CpSolver(), gates, planes)

Status do solucionador: INFEASIBLE
Sem solucao
